In [1]:
#R version 3.6.3 (2020-02-29)
#loading library used in this work. Version were specified.
                     
library(fdapace)      #version 0.5.5
library(funData)      #version 1.3.5
library(MFPCA)        #version 1.3.6
library(raster)       #version 3.3.13
library(randomForest) #version 4.6.14
library(caret)        #version 6.0.86
library(rgdal)        #version 1.5.16

Registered S3 methods overwritten by 'tibble':
  method     from  
  format.tbl pillar
  print.tbl  pillar


Attaching package: ‘funData’


The following object is masked from ‘package:stats’:

    integrate


Loading required package: sp


Attaching package: ‘raster’


The following object is masked from ‘package:funData’:

    approxNA


randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.

Loading required package: lattice

Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:randomForest’:

    margin


The following object is masked from ‘package:funData’:

    ggplot


rgdal: version: 1.5-16, (SVN revision 1050)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.1.2, released 2020/07/07
Path to GDAL shared files: /usr/share/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 7.1.1, September 1st, 2020, [PJ_VERSION: 711]
Path to PROJ shared

In [2]:
# Define a function named 'classify' that takes two arguments: 'dataset' and 'id'
classify <- function(dataset, id){
    # Assign the value of 'dataset' to a new variable named 'tdp'
    tdp <- dataset
    
    # Count the number of unique values in the 'CLASS' column of 'tdp' and assign it to a variable named 'nc'
    nc <- length(unique(tdp$CLASS))
    # Count the number of columns in 'tdp' and assign it to a variable named 'c1'
    c1 <- length(colnames(tdp))
    # Print the value of 'c1' to the console
    message(c1)
    # Repeat the minimum frequency of the values in the 'CLASS' column of 'tdp' for 'nc' times and assign it to a variable named 'campione'
    campione <- rep(min(table(tdp$CLASS)), nc)
    # Create a sequence of numbers from 2 to the number of columns in 'tdp' minus 1, with a step size of 4, and assign it to a variable named 'subsets'
    subsets <- seq(2, ncol(tdp)-1, 4)
    # Set the random seed to 1000 for reproducibility
    set.seed(1000)
    # Create a control object for the recursive feature elimination (rfe) function using the random forest functions, the repeated cross-validation method, 10 folds, 5 repeats, and no verbose output, and assign it to a variable named 'ctrl'
    ctrl <- rfeControl(functions = rfFuncs,
                       method = "repeatedcv",
                       number = 10,
                       repeats = 5,
                       verbose = FALSE)
    
    # Apply the rfe function to the 'tdp' data frame, using all columns except the last one as predictors and the last column as the response, and selecting the optimal number of features from the 'subsets' vector, using the 'ctrl' object as the control argument, and assign the result to a variable named 'lmProfile'
    lmProfile <- rfe(x = tdp[, c(1:ncol(tdp)-1)], y = as.factor(tdp$CLASS),
                     sizes = subsets,
                     rfeControl = ctrl)
    
    # Extract the names of the best predictors from the 'lmProfile' object and assign them to a variable named 'pr'
    pr <- predictors(lmProfile)[1: lmProfile$bestSubset]
    # Print the value of 'pr' to the console
    message(pr)
    
    # Assign the value of 'id' to a new variable named 'nomeTS'
    nomeTS <- id
    # Set the random seed to 1000 for reproducibility
    set.seed(1000)
    # Create a control object for the train function using the repeated cross-validation method, 10 folds, 5 repeats, and assign it to a variable named 'ctrl'
    ctrl <- trainControl(method = "repeatedcv", number = 10, repeats = 5)    
    # Subset the 'tdp' data frame by selecting only the columns in 'pr' and the 'CLASS' column and assign it to a new variable named 'dataset'
    dataset <- tdp[, c(pr, 'CLASS')]
    # Create a sequence of numbers from 1 to the square root of the number of columns in 'dataset' and assign it to a variable named 'mtry'
    mtry <- 1:sqrt(ncol(dataset))
    # Create a grid of tuning parameters for the random forest model by varying the 'mtry' argument and assign it to a variable named 'tunegrid'
    tunegrid <- expand.grid(.mtry = mtry)
    # Train a random forest model using the 'dataset' data frame, with the 'CLASS' column as the response and the rest as predictors, using 1500 trees, the 'tunegrid' as the tuning parameter grid, the 'Accuracy' metric as the performance measure, the 'CLASS' column as the stratification variable, the 'ctrl' object as the control argument, and the 'campione' vector as the sample size for each class, and assign the result to a variable named 'rfDownsampled'
    rfDownsampled <- train(CLASS ~ ., data = dataset,
                             method = "rf",
                             ntree = 1500,
                             tuneGrid = tunegrid,
                             metric = "Accuracy",
                             strata = dataset$CLASS,
                             trControl = ctrl,
                             sampsize = campione)
    # Return the value of 'rfDownsampled' from the function
    return(rfDownsampled)
}


In [3]:

# Define a function named 'finalMFPCA' that takes one argument: 'formula'
finalMFPCA <- function(formula){
    # Concatenate the letter 'F' with the value of 'formula' and assign it back to 'formula'
    formula <- paste("F", formula, sep='')
    # Concatenate the file path of the selected time series with the value of 'formula' and the suffix '_TSsel_N.txt' and assign it to a variable named 'path1'
    path1 <- paste('/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Smoothed_Time_Series/Selected/', formula, '_TSsel_N.txt', sep='')
    # Read the data from the file 'path1' as a delimited file with space as the separator, double quote as the quote character, and dot as the decimal point, and assign it to a variable named 'TS'
    TS <- read.delim(path1, header = TRUE, sep = " ", quote = "\"", dec = ".")
    # Convert the 'TS' data frame to a matrix and then to a vector and assign it back to 'TS'
    TS <- as.vector(as.matrix(TS))
    
    # Concatenate the file path of the time series with the value of 'formula' and the value of 'TS' and assign it to a variable named 'listOfFile'
    listOfFile <- paste("/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Smoothed_Time_Series/", formula, TS, sep = '/')
    
    # Create an empty list and assign it to a variable named 'lista1'
    lista1 <- list()
    # Create a counter variable and assign it to zero
    i <- 0
    
    # Loop over the values in the 'listOfFile' vector
    for (var in listOfFile) {
        # Print the current value to the console
        print(var)
        # Increment the counter by one
        i <- i + 1    
        # Assign the current value to a variable named 'fileName'
        fileName <- paste(var, sep="/")
        # Check if the 'fileName' is not an existing directory
        if (!dir.exists(fileName)) {
            # Read the data from the file 'fileName' as a delimited file with space as the separator, double quote as the quote character, and dot as the decimal point, and assign it to a variable named 'ts'
            ts <- read.delim(fileName, header = TRUE, sep = " ", quote = "\"", dec = ".")
            # Subset the 'ts' data frame by selecting only the rows from 5 to the end
            ts <- ts[5:nrow(ts),]
            # Convert the columns of the 'ts' data frame to numeric values and assign it back to 'ts'
            ts = as.data.frame(sapply(ts, function(x) as.numeric(as.character(x))))
            # Create a variable name by concatenating the string 'ts' with the value of the counter and assign the value of 'ts' to it
            assign(paste('ts', i, sep = ''), ts)
            # Convert the 'ts' data frame to a matrix and assign it to a variable named 'ts'
            ts <- as.matrix(ts)
            # Create a functional data object from the 'ts' matrix with the argument values from 1 to 52 and assign it to a variable named 'ts.f'
            ts.f <- funData(argvals = 1:52, X = (t(ts)))
            # Create a variable name by concatenating the string 'tsf' with the value of the counter and assign the value of 'ts.f' to it
            assign(paste('tsf', i, sep = ''), ts.f)
            # Add the value of 'ts.f' to the 'lista1' list at the position of the counter
            lista1[[i]] <- assign(paste('tsf', i, sep = ''), ts.f)
        }
    }
    
    # Create a multivariate functional data object from the 'lista1' list and assign it to a variable named 'm.funzioni'
    m.funzioni <- multiFunData(lista1)
    # Source the file '/home/data/PESARESI SIMONE/Pesaresi_et_al_2023/Functions_R/FPCAmulti.R' to load the function 'FPCAmulti'
    source("/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Functions_R/FPCAmulti.R")
    # Apply the function 'FPCAmulti' to the 'm.funzioni' object, the 'TS' vector, and the string 'N' and assign the result to a variable named 'scores'
    scores <- FPCAmulti(m.funzioni, TS, 'N')
    # Convert the 'scores' matrix to a data frame and assign it back to 'scores'
    scores <- as.data.frame(scores)
    
    # Add a new column named 'CLASS' to the 'scores' data frame with the values from the 'CLASS1' column of the 'pu' object
    scores$CLASS <- pu$CLASS1  # Note: 'pu' object is not defined in the given code snippet
    
    # Return a list with two components: 'scores' and 'curve' from the function
    return(list(scores = scores, curve = lista1))
}


In [4]:
# Define the file path of the shapefile
file.shape <- '/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Shapefile/TD_CONERO.shp'

# Read the shapefile into a spatial object
pu <- shapefile(file.shape)

# Define the file path of the smoothed time series
pathTS <- "/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Smoothed_Time_Series/F"



In [5]:
# Assign the string '01' to a variable named 'formula'
formula <- '01'
# Apply the function 'finalMFPCA' to the value of 'formula' and assign the result to a variable named 'MFPCA'
MFPCA <- finalMFPCA(formula)


[1] "/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Smoothed_Time_Series//F01/F01_06_02_00_00_10_BIL.txt"
[1] "/home/data/PESARESI SIMONE/Pesaresi_et_al_2024/Smoothed_Time_Series//F01/F01_07_02_00_00_10_BIL.txt"


Warning message in MFPCA(m.funzioni, M = 36, approx.eigen = FALSE, uniExpansions = lista.exp, :
“Function MFPCA: total number of univariate basis functions is smaller than given M. M was set to 20.”


In [11]:
str(MFPCA)
head(MFPCA[[1]])

List of 2
 $ scores:'data.frame':	175 obs. of  21 variables:
  ..$ V1   : num [1:175] -3494 -3360 -3867 -6444 -9610 ...
  ..$ V2   : num [1:175] 2860 282 539 3177 -312 ...
  ..$ V3   : num [1:175] -666 -1103 -403 -618 1910 ...
  ..$ V4   : num [1:175] -102.4 46.3 159.3 238 -1212.1 ...
  ..$ V5   : num [1:175] -484 -290 -786 -423 -363 ...
  ..$ V6   : num [1:175] -658 -764 -961 -499 -615 ...
  ..$ V7   : num [1:175] 65.5 159.1 14.5 99.5 -449.5 ...
  ..$ V8   : num [1:175] 34.04 -5.52 62.97 -8.97 221.23 ...
  ..$ V9   : num [1:175] 56.8 71.3 83.9 109.3 128.1 ...
  ..$ V10  : num [1:175] -27.3 -44.8 -167 76.3 161.3 ...
  ..$ V11  : num [1:175] 74.59 118.61 49 -1.06 -95.47 ...
  ..$ V12  : num [1:175] -32.8 55.1 71 41.9 94.7 ...
  ..$ V13  : num [1:175] -1.63 34.44 71.2 -62.78 64.7 ...
  ..$ V14  : num [1:175] 2.65 3.62 47.46 33.03 -25.55 ...
  ..$ V15  : num [1:175] 25.6 8.44 15.89 -2.25 27.62 ...
  ..$ V16  : num [1:175] 1.36 35.9 17.57 -9.78 69.62 ...
  ..$ V17  : num [1:175] 3.03 22.53

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,CLASS
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
V1,-3494.4281,2859.8652,-665.8759,-102.40138,-484.15280,-657.9288,65.47607,34.044059,56.76107,-27.28124,⋯,-32.81262,-1.630395,2.650221,25.604306,1.360362,3.029219,-22.259624,-7.244664,6.6027417,L3
V2,-3359.9248,281.5249,-1102.6663,46.29861,-289.52966,-763.8315,159.10266,-5.516437,71.25599,-44.77452,⋯,55.10268,34.435232,3.622388,8.437583,35.895617,22.533607,18.076858,17.210980,4.4751382,L3
V3,-3866.8597,538.8102,-403.1808,159.30495,-785.68485,-960.8600,14.51092,62.974132,83.93550,-166.96036,⋯,70.98285,71.196218,47.464304,15.894355,17.565163,10.494987,-2.948256,12.554137,12.0913820,L3
V4,-6443.8723,3177.2618,-618.4987,238.04662,-422.77324,-498.7878,99.53115,-8.972697,109.31366,76.34561,⋯,41.87209,-62.777034,33.034509,-2.251990,-9.777143,25.392704,-29.454228,-3.206344,-1.3808613,L3
V5,-9610.0078,-312.0745,1909.6197,-1212.13920,-363.28368,-614.9705,-449.49280,221.230607,128.14998,161.30189,⋯,94.69164,64.697064,-25.547148,27.615042,69.622877,14.112109,-4.982422,14.889432,4.9293633,L3
V6,-769.5357,-4164.1139,-528.8096,594.81975,-25.36103,-543.5062,284.38163,-116.620934,152.92597,-70.02799,⋯,63.54418,-61.598073,7.260938,-7.898849,-17.433672,-5.053091,41.831478,15.449142,0.9149901,L4


In [6]:
# Apply the function 'classify' to the 'scores' component of the 'MFPCA' object and the value of 'formula' and assign the result to a variable named 'model'
model <- classify(MFPCA[['scores']],formula)

# Print the value of 'model' to the console
model

# Compute and print the confusion matrix for the 'model' object
confusionMatrix(model)

# Compute and print the variable importance for the final model in the 'model' object
varImp(model$final)


21

V1V3V4V5V7V6



Random Forest 

175 samples
  6 predictor
  4 classes: 'L3', 'L4', 'O3', 'R' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 5 times) 
Summary of sample sizes: 157, 159, 158, 157, 158, 156, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
  1     0.8349862  0.7512827
  2     0.8254412  0.7391239

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 1.

Cross-Validated (10 fold, repeated 5 times) Confusion Matrix 

(entries are percentual average cell counts across resamples)
 
          Reference
Prediction   L3   L4   O3    R
        L3 38.1  5.4  4.2  2.6
        L4  2.2 15.2  0.0  1.1
        O3  0.2  0.0  3.2  0.0
        R   0.1  0.6  0.0 27.1
                            
 Accuracy (average) : 0.8354


,Overall
,<dbl>
V1,8.457338
V3,6.993768
V4,6.481464
V5,5.818205
V7,5.764917
V6,5.484308
